In [1]:
import torchvision
import numpy as np
import matplotlib
from src import preprocess, activation_functions, initializers, layer, loss_functions, optimizers, FFNN, training, wandblogger



In [2]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False)

In [3]:
import numpy as np
from src import preprocess

validation_percent = 0.2
one_hot_encoder = preprocess.OneHotEncoder()


train_size = int((1 - validation_percent) * len(train_dataset))
validation_size = len(train_dataset) - train_size

x_train = train_dataset.data[:train_size].reshape(train_size, -1)
targets_train = np.array(train_dataset.targets[:train_size])
y_train = one_hot_encoder.transform(targets_train)

x_valid = train_dataset.data[train_size:].reshape(validation_size, -1)
targets_valid = np.array(train_dataset.targets[train_size:])
y_valid = one_hot_encoder.transform(targets_valid)

x_test = test_dataset.data.reshape(len(test_dataset), -1)
targets_test = np.array(test_dataset.targets)
y_test = one_hot_encoder.transform(targets_test)

print(type(x_train), type(y_train))
print(x_train.shape, y_train.shape)

<class 'torch.Tensor'> <class 'numpy.ndarray'>
torch.Size([48000, 784]) (48000, 10)


/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/ipykernel_10483/1812696027.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_train = np.array(train_dataset.targets[:train_size])
/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/ipykernel_10483/1812696027.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_valid = np.array(train_dataset.targets[train_size:])
/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/ipykernel_10483/1812696027.py:20: DeprecationWarn

In [4]:
#find min and max value of y_train
print("y_train min:", np.min(y_train), "y_train max:", np.max(y_train))

y_train min: 0.0 y_train max: 1.0


In [5]:
from src import (
    FFNN,
    layer,
    activation_functions,
    initializers,
    loss_functions,
    optimizers,
    batchnorm,
    dropout,
)


activation_function = activation_functions.ReLU()
identity_activation = activation_functions.Identity()

weight_init = initializers.HeInitializer()
bias_init = initializers.ConstantInitializer(0.0)

loss_fn = loss_functions.CrossEntropyLoss()
optimizer = optimizers.SGD(learning_rate=0.01)

layers = [
    layer.Layer(
        input_size= 28 * 28,
        output_size=128,
        activation_function=activation_functions.Identity(),
        weight_initializer=weight_init,
        bias_initializer=bias_init,
    ),
    batchnorm.BatchNorm(128),
    activation_functions.ReLU(),
    dropout.Dropout(0.5),
    layer.Layer(
        input_size=128,
        output_size=64,
        activation_function=activation_functions.Identity(),
        weight_initializer=weight_init,
        bias_initializer=bias_init,
    ),
    batchnorm.BatchNorm(64),
    activation_functions.ReLU(),
    dropout.Dropout(0.5),
    layer.Layer(
        input_size=64,
        output_size=10,
        activation_function=activation_functions.Identity(),
        weight_initializer=weight_init,
        bias_initializer=bias_init,
    ),
]

model = FFNN.FFNN(layers=layers, loss_function=loss_fn, optimizer=optimizer)

In [6]:
# activation_func = activation_functions.ReLU()
# loss_func = loss_functions.CrossEntropyLoss()
# optimizer = optimizers.SGD()
# W_initializers = initializers.HeInitializer()
# B_initializer = initializers.ConstantInitializer(0)
# input_size = x_train.shape[1]
# hidden_size = 392
# output_size = 10

# Input_layer = layer.Layer(input_size=input_size, output_size=output_size, activation_function=activation_func, weight_initializer=W_initializers, bias_initializer=B_initializer)
# layers = [Input_layer]
# FeedForward = FFNN.FFNN(layers=layers, loss_function=loss_func, optimizer=optimizer)


In [7]:
epochs = 10
batch_size = 32
training.Trainer(model, loss_function=loss_fn, optimizer=optimizer).train(X=x_train, y=y_train, x_val=x_valid, y_val=y_valid, epochs=epochs, batch_size=batch_size, shuffle=True)

Epoch 1/10 - Loss: 2.2694 - Val Loss: 1.9262
Epoch 2/10 - Loss: 1.8292 - Val Loss: 1.7061
Epoch 3/10 - Loss: 1.6635 - Val Loss: 1.5747
Epoch 4/10 - Loss: 1.5503 - Val Loss: 1.4559
Epoch 5/10 - Loss: 1.4624 - Val Loss: 1.3761
Epoch 6/10 - Loss: 1.3922 - Val Loss: 1.3190
Epoch 7/10 - Loss: 1.3305 - Val Loss: 1.2513
Epoch 8/10 - Loss: 1.2851 - Val Loss: 1.2111
Epoch 9/10 - Loss: 1.2402 - Val Loss: 1.1638
Epoch 10/10 - Loss: 1.2017 - Val Loss: 1.1350


{'loss': [np.float64(2.2693613224445315),
  np.float64(1.829248276843451),
  np.float64(1.6634946924603506),
  np.float64(1.55026919148624),
  np.float64(1.462418859144988),
  np.float64(1.3921571580897907),
  np.float64(1.3304691432965523),
  np.float64(1.285116246710045),
  np.float64(1.2401656917347523),
  np.float64(1.201739082682527)],
 'val_loss': [np.float64(1.9262041760816282),
  np.float64(1.7061471132899082),
  np.float64(1.5746507461241164),
  np.float64(1.455940003760176),
  np.float64(1.376061352397132),
  np.float64(1.3190252421950643),
  np.float64(1.2513063135969988),
  np.float64(1.211067122459857),
  np.float64(1.1638420331349273),
  np.float64(1.1350155781412419)]}

In [8]:
accuracy = loss_functions.Accuracy() 
#accuracy
y_preds = model.forward_pass(x_test)
acc = accuracy.compute(y_test, y_preds)
print(f"Test accuracy: {acc * 100:.2f}%")

Test accuracy: 58.95%
